15分でできる日本語Word2Vec https://qiita.com/makaishi2/items/63b7986f6da93dc55edd

In [1]:
# zipファイルダウンロード
url = 'https://www.aozora.gr.jp/cards/000148/files/794_ruby_4237.zip'
zip = 'data/794_ruby_4237.zip'
import urllib.request
urllib.request.urlretrieve(url, zip)

# ダウンロードしたzipの解凍
import zipfile
with zipfile.ZipFile(zip, 'r') as myzip:
    myzip.extractall()
    # 解凍後のファイルからデータ読み込み
    for myfile in myzip.infolist():
        # 解凍後ファイル名取得
        filename = myfile.filename
        # ファイルオープン時にencodingを指定してsjisの変換をする
        with open(filename, encoding='sjis') as file:
            text = file.read()

# ファイル整形
import re
# ヘッダ部分の除去
text = re.split('\-{5,}',text)[2]
# フッタ部分の除去
text = re.split('底本：',text)[0]
# | の除去
text = text.replace('|', '')
# ルビの削除
text = re.sub('《.+?》', '', text)
# 入力注の削除
text = re.sub('［＃.+?］', '',text)
# 空行の削除
text = re.sub('\n\n', '\n', text) 
text = re.sub('\r', '', text)

# 整形結果確認

# 頭の100文字の表示 
print(text[:100])
# 見やすくするため、空行 
print()
print()
# 後ろの100文字の表示 
print(text[-100:])


一
　うとうととして目がさめると女はいつのまにか、隣のじいさんと話を始めている。このじいさんはたしかに前の前の駅から乗ったいなか者である。発車まぎわに頓狂な声を出して駆け込んで来て、いきなり肌をぬい


評に取りかかる。与次郎だけが三四郎のそばへ来た。
「どうだ森の女は」
「森の女という題が悪い」
「じゃ、なんとすればよいんだ」
　三四郎はなんとも答えなかった。ただ口の中で迷羊、迷羊と繰り返した。




In [2]:
# Janomeのロード
from janome.tokenizer import Tokenizer

# Tokenizerインスタンスの生成 
t = Tokenizer()

# テキストを引数として、形態素解析の結果、名詞・動詞・形容詞(原形)のみを配列で抽出する関数を定義 
def extract_words(text):
    tokens = t.tokenize(text)
    return [token.base_form for token in tokens 
        if token.part_of_speech.split(',')[0] in['名詞', '動詞']]

#  関数テスト
# ret = extract_words('三四郎は京都でちょっと用があって降りたついでに。')
# for word in ret:
#    print(word)

# 全体のテキストを句点('。')で区切った配列にする。 
sentences = text.split('。')
# それぞれの文章を単語リストに変換(処理に数分かかります)
word_list = [extract_words(sentence) for sentence in sentences]

# 結果の一部を確認 
for word in word_list[0]:
    print(word)

一
する
目
さめる
女
隣
じいさん
話
始める
いる


sentences=None, 
corpus_file=None, 
vector_size=100, 
alpha=0.025, 
window=5, 
min_count=5, 
max_vocab_size=None, 
sample=0.001, 
seed=1, 
workers=3, 
min_alpha=0.0001, 
sg=0, 
hs=0, 
negative=5, 
ns_exponent=0.75, 
cbow_mean=1, 
hashfxn=<built-in function hash>, 
epochs=5, 
null_word=0, 
trim_rule=None, 
sorted_vocab=1, 
batch_words=10000, 
compute_loss=False, 
callbacks=(), 
comment=None, 
max_final_vocab=None)

In [7]:
# Word2Vecライブラリのロード
from gensim.models import word2vec

# size: 圧縮次元数
# min_count: 出現頻度の低いものをカットする
# window: 前後の単語を拾う際の窓の広さを決める
# iter: 機械学習の繰り返し回数(デフォルト:5)十分学習できていないときにこの値を調整する
# model.wv.most_similarの結果が1に近いものばかりで、model.dict['wv']のベクトル値が小さい値ばかりの 
# ときは、学習回数が少ないと考えられます。
# その場合、iterの値を大きくして、再度学習を行います。

# 事前準備したword_listを使ってWord2Vecの学習実施
# print(help(word2vec.Word2Vec()))
model = word2vec.Word2Vec(word_list, vector_size=100,min_count=5,window=5,epochs=100)

In [8]:
print(model.__dict__['wv']['世間'])

[-0.2514574   0.5983756  -0.1054646  -0.06951263 -0.4410906  -0.53059053
 -0.23402226  0.29629037 -0.10588566  0.2983036   0.33217373 -0.40734458
  0.345004    0.6543715   0.41360125 -0.07991505 -0.17177987 -0.20790139
 -0.07939453 -0.53092706 -0.9385529   0.44570765  0.3491775  -0.94380504
  0.28177077  0.01750118 -0.34375364 -0.27269652 -0.00292351 -0.7102562
  0.47592166 -0.36076626  0.10864413 -0.28852105 -0.47496647  0.508098
 -0.61526084 -0.30691984 -0.24021763  0.06906699  0.5480436  -0.16006823
 -0.44741383 -0.6408411  -0.5850065   0.80511105 -0.14887643 -0.39739117
  0.6046842  -0.37735268  0.65816826 -0.37087408  0.7283094  -0.3201044
  0.23423342 -0.80463755  0.47638243  0.16982616 -0.5547485   0.58759356
 -0.7584049   0.2320579  -0.80850285 -0.5472746  -0.98315924  0.2319699
 -0.10905959 -0.6251506   0.12024964  0.4231289  -0.46065265 -0.5022147
 -0.3022688  -0.08488819  0.47481737 -0.17572248 -0.08048092  0.49595952
  0.29220933  0.27159315 -0.87362033  0.20104706 -0.29757

In [9]:
# 結果の確認2
# 関数most_similarを使って「世間」の類似単語を調べます 
ret = model.wv.most_similar(positive=['世間']) 
for item in ret:
    print(item[0], item[1])

喝采 0.5834517478942871
聞こえる 0.5745024085044861
外国 0.5421279072761536
自己 0.5368075370788574
社会 0.49744874238967896
決心 0.4789580702781677
賛成 0.463409423828125
有す 0.46222949028015137
堪える 0.4474026560783386
彼ら 0.4430335462093353
